# Translacja
Chodzi o to żeby przekształcić tekst, aby dało się go wyrenderować przy pomocy mojego specjalnego fontu dla systemu stenografii.
Założeniem systemu stenografii jest to, żeby nie trzeba było odrywać pióra przy pisaniu wyrazu.

Idea systemu:
1. Pisze się wokół linii środkowej ciągnąc kreskę do linii górnej lub dolnej i prawie zawsze (oprócz końca wyrazu) wracając do linii środkowej
1. Kreski mają różne (proste) kształy (w sumie 11)
1. Spółgłoski oznaczane są kreską od linii środkowej w górę lub w dół (różne spółgłoski dla lustrzanej kreski idącej w górę i w dół)
1. Samogłoski oznaczane są kreską od linii górnej lub dolnej do środkowej (ta sama samogłoska może być napisana wracając z góry jak i z dołu po przez lustrzane odbicie)
1. Jeśli występują po sobie dwie spółgłoski lub dwie samogłoski, to między nimi następuje powrót przy pomocy linii pionowej
1. Na końcu spółgłoski może dodany być jeden z dwóch akcentów (kreska i kółko/pętelka)
1. Na końcu samogłoski może dodany być akcent (mała skośna linia i powrót) który zmienia `e` na `ę`, `a` na `ą`
1. Generalnie to można pisać też samogłoski poziomo, na linii środkowej, co jest przydatne gdy samogłoska następuje po samogłosce. Jest to obsłużone przez specjalne ligatury. Póki co tylko `ie` i `ia`.

## Transkrypcja

Pierwszą fazą jest podzielenie tekstu na głoski.

In [1]:
import re

space_pattern = re.compile(r'\s+')
illegal_pattern = re.compile(r'[0-9]+')
def cleanup(t):
    return re.sub(space_pattern, ' ', re.sub(illegal_pattern, '', t.lower()))

cleanup("Nie! 117  dąsać się-pROszę.!")

'nie! dąsać się-proszę.!'

In [2]:
split_pattern = re.compile(r'(sz|cz|rz|dzi|dz|dź|dż|ch|si|ci|ni)')
def dziel(t):
    pos = 0
    for m in re.finditer(split_pattern, t):
        s = m.span()
        if m.start() != pos:
            for c in t[pos:m.start()]:
                yield c
        yield m[1]
        pos = m.end()
    if pos < len(t):
        for c in t[pos:]:
            yield c

print(list(dziel('zaszczute rzeźnickie dżdżownice dźwięczą dzisiaj dzdz ')))

['z', 'a', 'sz', 'cz', 'u', 't', 'e', ' ', 'rz', 'e', 'ź', 'ni', 'c', 'k', 'i', 'e', ' ', 'dż', 'dż', 'o', 'w', 'ni', 'c', 'e', ' ', 'dź', 'w', 'i', 'ę', 'cz', 'ą', ' ', 'dzi', 'si', 'a', 'j', ' ', 'dz', 'dz', ' ']


Z `ś` + `si`, `ć` + `ci` oraz `ń` + `ni` nie jest oczywiste której użyć. Jeśli po niej jest samogłoska, to chciłoby się uzyć `ś` a jeśli spółgłoska, to lepiej osobno `s` `i` (ze względu na komplikacje przy powtórzonych samogłoskach).
Podobną regułę moza przyjąć dla `dź` + `dzi`.

In [3]:
def czy_samlgloska(s):
    return s in ['a','i','e','o','y','u','ę','ą','ó']

def czy_spolgloska(s):
    return str.isalpha(s) and not czy_samlgloska(s)

In [4]:
def popraw(iter):
    prev = next(iter)
    for g in iter:
        if prev in ['si', 'ci', 'ni', 'dzi'] and czy_spolgloska(g):
            yield prev[:-1]
            yield 'i'
        else:
            yield prev
        prev = g
    yield prev

print(list(popraw(dziel('sieć i sito dzisiaj dziadek ciepło ciska'))))

['si', 'e', 'ć', ' ', 'i', ' ', 's', 'i', 't', 'o', ' ', 'dz', 'i', 'si', 'a', 'j', ' ', 'dzi', 'a', 'd', 'e', 'k', ' ', 'ci', 'e', 'p', 'ł', 'o', ' ', 'c', 'i', 's', 'k', 'a']



Utożsamiane są niektóre głoski. Mamy podstawienia:
- `ch` -> `h`
- `rz` -> `ż`
- `dzi` -> `dź`
- `rz` -> `ż`
- `ó` -> `u`
- `si` -> `ś`
- `ci` -> `ć`
- `ni` -> `ń`

Akcent pierwszy (pętelka) daje podstawienia (`1` oznacza górną formę akcentu, `3` oznacza dolną)
- `ś` -> `s1`
- `ć` -> `c1`
- `dź` -> `d3`
- `ź` -> `z3`
- `ń` -> `n3`

Akcent drugi (kreska w górę/dół oznaczana przez `2`/`4`) daje:
- `sz` -> `s2`
- `cz` -> `c2`
- `dż` -> `d4`
- `ż` -> `z4`

In [5]:
podstawienia = {
    'ch': 'h',
    'rz': 'z4',
    'dzi': 'd3',
    'rz': 'z4',
    'ó': 'u',
    'si': 's1',
    'ci': 'c1',
    'ni': 'n3',
    'ś': 's1',
    'ć': 'c1',
    'dź': 'd3',
    'ź': 'z3',
    'ń': 'n3',
    'sz': 's2',
    'cz': 'c2',
    'dż': 'd4',
    'ż': 'z4'
}

def podstaw(iter):
    for c in iter:
        if c in podstawienia:
            yield podstawienia[c]
        else:
            yield c

Spółgłoski dzielą się na górne i dolne.

In [6]:
gorne = {
    'm',
    's',
    't',
    'l',
    'p',
    'k',
    'w',
    'r',
    'c',
    'dz',
    'x'
}

dolne = {
    'n',
    'z',
    'd',
    'ł',
    'b',
    'g',
    'f',
    'j',
    'h',
    'v',
    'q'
}

Po górnej spółgłosce może występować tylko górna samogłoska (oznaczona upper-case), a po dolnej - dolna.

Jeśli po spółgłosce jest spółgłoska, to między nimi musi znaleźć się linia wracająca do środkowej linii (górna `\` lub dolna `/`).

Tymczasowo podobnie dla samogłosek, póki nie są obsługiwane poziome formy samogłosek.

In [7]:
akc_pattern = re.compile(r'[1234]$')

def bez_akcentu(c):
    return re.sub(akc_pattern, '', c)

def klasyfikuj(c):
    cba = bez_akcentu(c)
    sam = czy_samlgloska(c[0])
    sp = czy_spolgloska(c[0])
    inna = not sam and not sp
    gorna = (sp and cba in gorne) or inna
    return sam, sp, inna, gorna

def ligatura(prev, c):
    return (prev, c) in [('i','e'), ('i','a')]

def polacz(iter):
    prev = next(iter)
    prev_sam, prev_sp, prev_inna, prev_gorna = klasyfikuj(prev)
    next_up = prev_sam
    for c in iter:
        sam, sp, inna, gorna = klasyfikuj(c)
        go_up = next_up
        next_up = False
        if sam and prev_sam:
            if ligatura(prev, c):
                next_up = go_up # ten sam case
            else:
                # idziemy w górę
                prev = prev + '\\'
                next_up = True
        elif sam and prev_gorna:
            next_up = True
        elif sp and prev_sp:
            if prev_gorna:
                prev = prev + '\\'
            else:
                prev = prev + '/'
        # emit before going to next
        if go_up:
            yield prev.upper()
        else:
            yield prev
        prev = c
        prev_sam, prev_sp, prev_inna, prev_gorna = sam, sp, inna, gorna
    # emit last one
    if next_up:
        yield prev.upper()
    else:
        yield prev
        
print(list(polacz(['dz','i', 'k', 'i'].__iter__())))

['dz', 'I', 'k', 'I']


Wygląd akcentu `ą` i `ę` zależy od tego czy jest to samogłoska górna czy dolne. Dlatego dopiero teraz możemy o nim zdecydować.

In [8]:
ogonki = {
    'Ą': 'A5',
    'Ę': 'E5',
    'ą': 'a6',
    'ę': 'e6'
}

def ogonkuj(iter):
    for c in iter:
        if c in ogonki:
            yield ogonki[c]
        else:
            yield c

In [9]:
def konwertuj(s):
    return ogonkuj(polacz(podstaw(popraw(dziel(cleanup(s))))))

''.join(konwertuj('i tak i dzisiejszy dzień zaczął się bardzo nietypowo'))

'I tAk I dzIs1Ej/s2Y d3en3 zac2A5ł s1E5 bar\\dzO n3etYpOwO'

In [10]:
with open('8119-0.txt') as f:
    c = 1
    for line in f:
        print(''.join(konwertuj(line)))
        if c > 100:
            break
        c += 1

s1Er\pIEn3 nawIEdzEn3e p\tAkI mAnekIny t\rAk\tAt O mAnekInah Al\bo w\tUrA 
 
k\s1E5ga rOdzAju t\rAk\tAt O mAnekInah c1A5g dal\s2Y t\rAk\tAt O mAnekInah 
 
dokOn3/c2En3e nem\rOd pAn pAn kArOl s\k\lEpY cYnamOnowE UlIcA k\rOkOdylI 
 
kArAkOny wIhurA noc wIEl\kIEgo sEzonu 
 
 
 
 
 
 
 
 
 
 s1Er\pIEn3 
 
 
 
 
 
 
 
 w lIp\cU Oj/c1Ec mUj wYjez4/d4ał do wUd I zos\tAwIAł m\n3e z mAt\kA5 I 
 
s\tAr\s2Ym b/rAtEm na pAs\t\wE5 białyh Od z4arU I Os2OłamIAja6cYh d/n3 lEt\nih. 
 
wEr\tOwAlIs1\mY, Oduz4en3 s1\wIAt\łem, w tEj wIEl\kIEj k\s1E5dzE wAkAc\ji, k\tUrEj 
 
w\s2Ys\t\kIE kAr\tY pAłały Od b/lAs\kU I mIAły na d/n3e s\łod/kI do Om\d/lEn3a 
 
mI\A5z4/s2 z/łotYh g/rUs2Ek. AdelA w\rAcAła w s1\wIEt\lIs\tE pOrAn/kI, jak pOmOna z 
 
Og/n3a d/n3a rOz/z4ag/wI\Onego, wYsYpUja6c z kOs2YkA bar\w\na6 UrOde6 s\łon3/cA - 
 
l\s1\n3a6cE, pEł/ne wOdy pOd p\z4ej/z4ys\tA5 s\kUr\kA5 c2ErEs1\n3e, tAjem\nic2E, c2Ar\ne 
 
wIs1\n3e, k\tUrYh wOn3 p\z4ek\rAc2Ała tO, cO zis2\c2Ało s1E5 w s\mAkU; mOrElE, w 
 
k\tUrYh mI\